In [1]:
# Code Implemented by,
# Name: Rohith Ganesan
# Id: 20553375
# Mail: psxrg10@nottingham.ac.uk


import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.table import Table

matplotlib.use('Agg')



# Iterative Policy Evalution no Smal Gridworld
# Gridworld is a common testbed environment for new RL algorithms.
# We consider a small Gridsworld, a 4x4 grid of celLs,
# where the northmost-westmost cell and the southmost-eastmost cell are terminal state 
# The agent can move between different cells.

WORLD_SIZE = 4

#Left, up, right, down
ACTIONS = [np.array([0, -1]), np.array([-1, 0]), np.array([0, 1]), np.array([1, 0])]
ACTION_PROB = 0.25

def is_terminal(state):
    x, y = state
    return (x == 0 and y == 0) or (x == WORLD_SIZE - 1 and y == WORLD_SIZE - 1)

def step(state, action):
    if is_terminal(state):
        return state, 0
    next_state = (np.array(state) + action).tolist()
    x, y = next_state
    if x < 0 or x >= WORLD_SIZE or y < 0 or y >= WORLD_SIZE:
        next_state = state
    reward = -1
    return next_state, reward

def draw_image(image):
    fig, ax = plt.subplots()
    ax.set_axis_off()
    tb = Table(ax, bbox=[0, 0, 1, 1])
    nrows, ncols = image.shape
    width, height = 1.0 / ncols, 1.0 / nrows

    # Add cells
    for (i, j), val in np.ndenumerate(image):
        tb.add_cell(i, j, width, height, text=val, loc='center', facecolor='white')

    # Row and column Labels...
    for i in range(len(image)):
        tb.add_cell(i, -1, width, height, text=i+1, loc='right', edgecolor='none', facecolor='none')
        tb.add_cell(-1, i, width, height/2, text=i+1, loc='center', edgecolor='none', facecolor='none')

    ax.add_table(tb)

def compute_state_value(in_place=True, discount=1.0):
    new_state_values = np.zeros((WORLD_SIZE, WORLD_SIZE))
    iteration = 0
    while True:
        if in_place:
            state_values = new_state_values
        else:
            state_values = new_state_values.copy()
        old_state_values = state_values.copy()
        for i in range(WORLD_SIZE):
            for j in range(WORLD_SIZE):
                value = 0
                for action in ACTIONS:
                    (next_i, next_j), reward = step([i, j], action)
                    value += ACTION_PROB * (reward + discount * state_values[next_i, next_j])
                new_state_values[i, j] = value
        max_delta_value = abs(old_state_values - new_state_values).max()
        if max_delta_value < 1e-4:
            break
        iteration += 1
    return new_state_values, iteration

def figure_4_1():
    # While the author in the book (Richard S. Sutton, Andrew G Barto - Reinforcement Learning_ An Introduction, 2nd Edition-Bradford Books (2018)) suggests using in-place iterative policy evaluation,
    # Figure 4.1 actually uses out-of-place version.
    _, sync_iteration = compute_state_value(in_place=True)
    values, async_iteration = compute_state_value(in_place=False)
    draw_image(np.round(values, decimals=2))
    print('Synchronous: {} iterations'.format(sync_iteration))
    print('Asynchronous: {} iterations'.format(async_iteration))

if __name__ == '__main__':
    figure_4_1()
    plt.savefig('grid_world_plot.png')
    plt.close()


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Synchronous: 113 iterations
Asynchronous: 172 iterations
